### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
seed

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
%pylab inline

### ------------------------------------------------------------------------------------------------------------

In [ ]:
data_timeperiod = '15min'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

In [ ]:
system.train_set_end = 0.5 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 0.75 # percentage point specifying the validation set end point (1.0 means no test set)
# basically this is the data with the values above, which are like sliders determining the layout
# [|0.0| ...... train ............. |0.5| ............ val ............ |0.75| .............. test ............... |1.0|]

In [ ]:
feature_name = 'times in row'

In [ ]:
# Train classifier on train data
clf = train_classifier(LogisticRegression, data)

In [ ]:
s = MLClassifierStrategy(clf, list(data.filter(like='X')))
equity, pf, trades = backtest_ml_strategy(s, data)
pf

In [ ]:
# backtest results
plt.plot(equity);

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

In [ ]:
# Compute the profit factor for every candidate value
levels = [0,1,2,3,4,5,6,7,8,9,10]
pfs = []
nts = []
for l in tqdm(levels):
    pf, ntrades = compute_stats(data, filter_trades_by_feature(trades, data, featformat(feature_name), exact_value=l))
    pfs.append(pf)
    nts.append(len(ntrades))

In [ ]:
# Plot the results
plt.plot(levels, pfs);
plt.xlabel(feature_name);
plt.ylabel('Profit Factor');

In [ ]:
res = pd.DataFrame(data = np.hstack([ np.array(nts).reshape(-1,1),
                                      np.array(pfs).reshape(-1,1)]),
             index=np.array(levels),
             columns=['num trades', 'profit factor'])
res

In [ ]:
min_best_x = 2
max_best_x = 3

### ------------------------------------------------------------------------------------------------------------

In [ ]:
# Test on unseen data

In [ ]:
# Base test without the filter
s = MLClassifierStrategy(clf, list(data.filter(like='X')))
equity, pf, trades = backtest_ml_strategy(s, data, skip_val=1, skip_test=0)
# backtest results
plt.plot(equity);
pf

In [ ]:
# Test with the filter
equity, pf, trades = backtest_ml_strategy(s, data, skip_val=1, skip_test=0)
# filter stats
pf, trades = compute_stats(data, filter_trades_by_feature(trades, data, featformat(feature_name), min_value=min_best_x, max_value=max_best_x))
pf

In [ ]:
trades.head()

In [ ]:
plot(trades['profit'].cumsum());

In [ ]:
len(trades)